In [1]:
#!/usr/bin/env python3

%reload_ext autoreload
%autoreload 2

# System
import os
import sys
from tqdm.notebook import tqdm

# Maths
import numpy as np
import scipy.sparse as sps

# PCovR utilities
from regression import LR, KRR, SparseKRR, PCovR, KPCovR, SparseKPCovR
from decomposition import PCA, KPCA, SparseKPCA
from kernels import linear_kernel, gaussian_kernel, center_kernel
from tools import FPS, simple_split, CUR

# ASE
from ase.io import read, write

# SOAP
from rascal.representations import SphericalInvariants as SOAP

# Scikit learn
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.decomposition import PCA as skPCA
from sklearn.decomposition import KernelPCA as skKPCA

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt

# Make the plots look nicer
plot_parameters = {
    'lines.linewidth': 1.0,
    'lines.markersize': 2,
    'patch.linewidth': 1.0,
    'hatch.linewidth': 1.0,
    'axes.linewidth': 1.0,
    'xtick.top': True,
    'xtick.bottom': True,
    'xtick.direction': 'in',
    'xtick.minor.visible': True,
    'xtick.major.size': 4.0,
    'xtick.minor.size': 2.0,
    'xtick.major.pad': 5.0,
    'xtick.minor.pad': 5.0,
    'ytick.left': True,
    'ytick.right': True,
    'ytick.direction': 'in',
    'ytick.minor.visible': True,
    'ytick.major.size': 4.0,
    'ytick.minor.size': 2.0,
    'ytick.major.pad': 5.0,
    'ytick.minor.pad': 5.0   
}

for pp in plot_parameters.keys():
    mpl.rcParams[pp] = plot_parameters[pp]

In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/helfrech/.config/matplotlib/stylelib/cosmoLarge.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
# Read xyz files
s = read('/scratch/helfrech/Sync/GDrive/Projects/KPCovR/KernelPCovR/datasets/CSD-1000R.xyz', index=':5')

# Extract chemical shifts
cs = []
for ss in s:
    ss.wrap()
    cs.append(ss.arrays['CS_local'])

In [3]:
# Compute SOAPs (from librascal tutorial)
soap = SOAP(soap_type='PowerSpectrum',
           interaction_cutoff=3.5,
           max_radial=6,
           max_angular=6,
           gaussian_sigma_type='Constant',
           gaussian_sigma_constant=0.4,
           cutoff_smooth_width=0.5)

In [4]:
soap_rep = soap.transform(s)

In [5]:
X = soap_rep.get_features(soap)
Y = np.concatenate(cs)

In [6]:
# Train-Test split
f_train = 0.80
X_train, X_test, Y_train, Y_test = simple_split(X, Y, f_train)

In [7]:
# Center the data
X_mean = np.mean(X_train, axis=0)
Y_mean = np.mean(Y_train)

X_train -= X_mean
X_test -= X_mean
Y_train -= Y_mean
Y_test -= Y_mean

# FPS

In [8]:
# Select FPS components from train set
n_FPS = 20
idxs, d = FPS(X_train.T, n_FPS)
print(idxs)

[1476    0 1008 2268  294  280  196  378 1764  882   28  784  448  147
 1029  420   21  259 1680  924]


# CUR

In [9]:
idxs_c, idxs_r = CUR(X_train, n_col=20, n_row=0)
print(idxs_c)
print(idxs_r)

[1.47283101e-01 1.20690125e-05 2.17698032e-06 ... 6.37498863e-07
 4.26492822e-07 5.37821487e-07]
0.14728310140845316 2.4088258856653443e-15
[6.40790632e-36 3.43256972e-06 7.29051676e-07 ... 3.56219543e-06
 5.04861302e-06 2.46086107e-06]
0.1353274879475496 6.407906315462891e-36
[2.78370153e-37 2.25793049e-06 3.87758564e-07 ... 3.40657521e-05
 4.58949101e-05 2.55533714e-05]
0.09830447794727894 2.7837015260751047e-37
[5.94093411e-36 2.58761465e-06 8.47098104e-07 ... 3.63889658e-05
 4.38611177e-05 2.99289953e-05]
0.10068828861863695 3.0284872897086564e-37
[5.10296183e-36 6.08648569e-05 1.06752516e-05 ... 1.45772973e-07
 1.73951862e-06 3.52968205e-07]
0.12889986268650047 6.645294973613878e-40
[4.00664928e-37 2.52031979e-07 5.46520493e-09 ... 9.34233762e-06
 1.45722767e-05 9.19824938e-06]
0.062181215970150756 3.5959522017384006e-37
[2.70886185e-36 5.73504938e-07 2.38457221e-07 ... 6.28300797e-06
 4.65746917e-06 4.06856893e-06]
0.07074548493115465 7.391441813007418e-38
[3.04017947e-36 1.28488

# CUR from KPCovR notebook

In [10]:
k=1
nCUR = 20
print(nCUR)
A_copy = X_train.copy()

20


In [11]:
%time
(U, sig, V) = np.linalg.svd(A_copy)
pi = (V[:k]**2.0).sum(axis=0)
j = pi.argmax()

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.68 µs


In [12]:
%time
(U, sig, V) = sps.linalg.svds(A_copy,k)
pi = (V[:k]**2.0).sum(axis=0)
j = pi.argmax()

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 6.91 µs


In [13]:
v = A_copy[:,j]/np.sqrt(np.matmul(A_copy[:, j],A_copy[:, j]))

for i in range(A_copy.shape[1]):
    A_copy[:,i] -= v * np.dot(v,A_copy[:,i])

In [14]:
idxs = [j]

for n in tqdm(range(nCUR-1)):
    (U, sig, V) = sps.linalg.svds(A_copy,k)
    pi = (V[:k]**2.0).sum(axis=0)
    #pi[idxs] = 0 #####
    idxs.append(pi.argmax())
    
    v = A_copy[:,idxs[-1]]/np.sqrt(np.matmul(A_copy[:, idxs[-1]],A_copy[:, idxs[-1]]))

    for i in range(A_copy.shape[1]):
        A_copy[:,i] -= v * np.dot(v,A_copy[:,i])

In [15]:
idxs = np.asarray(idxs)
print(idxs)

[   0 1008 2268  196  294  147  448  434  840  420  784  924   14  672
  938 1022  378  945  441 2030]


# CUR-PCovR

In [58]:
idxs_c, idxs_r = CUR(X_train, Y_train, n_col=20, n_row=0, alpha=1.0)
print(idxs_c)
print(idxs_r)

[1.47283101e-01 1.20690125e-05 2.17698032e-06 ... 6.37498863e-07
 4.26492822e-07 5.37821487e-07]
0.1472831014084534 2.408825885644318e-15
[1.30785917e-35 3.43256972e-06 7.29051676e-07 ... 3.56219543e-06
 5.04861302e-06 2.46086107e-06]
0.13532748794754967 1.3078591679913825e-35
[2.40943259e-37 2.25793049e-06 3.87758564e-07 ... 3.40657521e-05
 4.58949101e-05 2.55533714e-05]
0.09830447794727887 2.4094325854199354e-37
[2.22754871e-36 2.58761465e-06 8.47098104e-07 ... 3.63889658e-05
 4.38611177e-05 2.99289953e-05]
0.10068828861863702 2.227548713799348e-36
[2.88404148e-35 6.08648569e-05 1.06752516e-05 ... 1.45772973e-07
 1.73951862e-06 3.52968205e-07]
0.12889986268650058 1.3951941274054764e-37
[3.56469655e-39 2.52031979e-07 5.46520493e-09 ... 9.34233762e-06
 1.45722767e-05 9.19824938e-06]
0.06218121597015027 3.5646965549273e-39
[3.12429617e-37 5.73504938e-07 2.38457221e-07 ... 6.28300797e-06
 4.65746917e-06 4.06856893e-06]
0.0707454849311546 2.363174862056862e-37
[8.46270578e-36 1.28488253e-

# CUR-PCovR from KPCovR notebook

In [46]:
def sorted_eig(mat, thresh=0.0, n=None):
    """
        Returns the eigenvalues and vectors sorted
        from largest to smallest eigenvalue
    """
    val, vec = np.linalg.eigh(mat)
    val = np.flip(val, axis=0)
    vec = np.flip(vec, axis=1)

    vec[:, val<thresh] = 0
    val[val<thresh] = 0

    return val[:n], vec[:, :n]


def get_Ct(X, Y, alpha=0.5, regularization=1e-6):
    """ Creates the PCovR modified covariance"""
    
    cov = np.matmul(X.T, X)
    v_C, U_C = sorted_eig(cov, thresh=regularization)
    
    Csqrt = np.matmul(np.matmul(U_C, np.diag(np.sqrt(v_C))), U_C.T)

    C_lr = np.matmul(np.linalg.pinv(cov, rcond=regularization), np.matmul(X.T,Y))
    C_lr = np.matmul(Csqrt, C_lr)
    C_lr = np.matmul(C_lr, C_lr.T) ###
    
    C_pca = cov
    
    C =  alpha*C_pca +  (1.0-alpha)*C_lr
   
    return C

def get_Ct2(X, Y, alpha=0.5, regularization=1e-6):                                                 
        
    cov = np.matmul(X.T, X)
    v_C, U_C = sorted_eig(cov, thresh=regularization)                                                 
    U_C = U_C[:, v_C>0]                                                                               
    v_C = v_C[v_C>0] 
                                                                                                      
    v_inv = np.array([np.linalg.pinv([[v]])[0][0] for v in v_C])                                      
    
    Csqrt = np.matmul(np.matmul(U_C, np.diag(np.sqrt(v_C))), U_C.T)                                   
    C_inv = np.matmul(np.matmul(U_C, np.diag(v_inv)), U_C.T)                                          
    
    C_lr = np.matmul(C_inv, np.matmul(X.T,Y))                                                         
    C_lr = np.matmul(Csqrt, C_lr)
    C_lr = np.matmul(C_lr, C_lr.T)                                                                    

    C =  alpha*cov +  (1.0-alpha)*C_lr                                                                

    return C

In [47]:
X_copy = X_train.copy()
Y_copy = Y_train.copy()

In [48]:
k = 1
alpha = 0.9999
nCUR = 10
Ct = get_Ct(X_copy/np.linalg.norm(X_copy), Y_copy/np.linalg.norm(Y_copy), alpha=alpha, regularization=1.0E-15)
Ct2 = get_Ct2(X_copy/np.linalg.norm(X_copy), Y_copy/np.linalg.norm(Y_copy), alpha=alpha, regularization=1.0E-15)

In [49]:
print(Ct)
print(Ct2)

[[ 5.88877060e-02 -3.30522999e-04 -8.09250880e-05 ...  2.19243409e-04
   1.99253626e-04  2.04802118e-04]
 [-3.30522999e-04  1.12119043e-04  1.05086061e-04 ...  1.00006193e-04
   9.86090876e-05  9.96945136e-05]
 [-8.09250880e-05  1.05086061e-04  1.02362140e-04 ...  1.00171082e-04
   9.93302196e-05  1.00058791e-04]
 ...
 [ 2.19243409e-04  1.00006193e-04  1.00171082e-04 ...  1.11726182e-04
   1.10223603e-04  1.09350098e-04]
 [ 1.99253626e-04  9.86090876e-05  9.93302196e-05 ...  1.10223603e-04
   1.14612312e-04  1.09839113e-04]
 [ 2.04802118e-04  9.96945136e-05  1.00058791e-04 ...  1.09350098e-04
   1.09839113e-04  1.10905547e-04]]
[[ 5.88877060e-02 -3.30522999e-04 -8.09250880e-05 ...  2.19243409e-04
   1.99253626e-04  2.04802118e-04]
 [-3.30522999e-04  1.12119043e-04  1.05086061e-04 ...  1.00006193e-04
   9.86090876e-05  9.96945136e-05]
 [-8.09250880e-05  1.05086061e-04  1.02362140e-04 ...  1.00171082e-04
   9.93302196e-05  1.00058791e-04]
 ...
 [ 2.19243409e-04  1.00006193e-04  1.0017108

In [50]:
v_Ct, U_Ct = sorted_eig(Ct)

pi = (U_Ct[:,:k]**2.0).sum(axis=1)
print(pi)
j = pi.argmax()

[1.47332808e-01 1.16587696e-05 2.00491075e-06 ... 7.36506640e-07
 5.08173440e-07 6.29105511e-07]


In [51]:
X_c = X_copy[:, [j]]

In [52]:
Q = np.linalg.solve(np.matmul(X_c.T,X_c),np.matmul(X_c.T,X_copy))
Q = np.linalg.cholesky(np.matmul(Q,Q.T))

T = np.matmul(X_c, Q)

In [53]:
v1 = np.linalg.pinv(np.matmul(T.T, T))
v1 = np.matmul(T, v1)
v1 = np.matmul(v1, T.T)

v2 = np.linalg.pinv(np.matmul(X_c.T, X_c))
v2 = np.matmul(X_c, v2)
v2 = np.matmul(v2, X_c.T)

In [54]:
Y_copy -= np.matmul(v2, Y_copy)

In [55]:
v = X_copy[:,j]/np.sqrt(np.matmul(X_copy[:, j],X_copy[:, j]))

for i in range(X_copy.shape[1]):
    X_copy[:,i] -= v * np.dot(v,X_copy[:,i])

In [56]:
idxs = [j]

for n in tqdm(range(nCUR-1)):
    
    try:
        Ct = get_Ct(X_copy, Y_copy, alpha=alpha)
    except:
        print(f"Only {n} features possible")
        break
        
    v_Ct, U_Ct = sorted_eig(Ct)
    
    pi = (U_Ct[:,:k]**2.0).sum(axis=1)
    print(pi)
    print(np.amax(pi), np.amin(pi))
    
    j=pi.argmax()
    idxs.append(j)
    
    X_c = X_copy[:, idxs]
    v = np.linalg.pinv(np.matmul(X_c.T, X_c))
    v = np.matmul(X_c, v)
    v = np.matmul(v, X_c.T)

    Y_copy -= np.matmul(v, Y_copy)
    
    v = X_copy[:,j]/np.sqrt(np.matmul(X_copy[:, j],X_copy[:, j]))

    for i in range(X_copy.shape[1]):
        X_copy[:,i] -= v * np.dot(v,X_copy[:,i])

[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.00039683]
0.0003968307873173912 0.00039681740198272574
[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.00039683]
0.0003968289816132657 0.0003968206049040105
[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.00039683]
0.0003968298966154304 0.0003968188600866042
[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.00039683]
0.0003968295932627368 0.0003968193782971142
[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.00039683]
0.0003968289998798138 0.00039681939431651187
[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.00039683]
0.0003968293309517867 0.0003968195883158087
[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.00039683]
0.0003968281873455863 0.00039681917010375527
[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.00039683]
0.0003968312486872693 0.00039681287132547246
[0.00039683 0.00039683 0.00039683 ... 0.00039683 0.00039683 0.000396

In [57]:
idxs = np.asarray(idxs)
print(idxs)

[   0  784  420  672 2268  280 1540  840  441  700]
